In [1]:
import utils
import torch
from torchvision import models, transforms
import torch.nn.functional as F
import torch.nn as nn
import json
import gzip

from utils import get_coarse_arrays, eliminate_elements_torch
from utils import ensamble_attack_coarse_classes_sum_full_v2

from PIL import Image, ImageEnhance
import os
from tqdm import tqdm

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#with open(os.path.join(res_path, 'results.json'), 'r') as json_file:
#    loaded_results = json.load(json_file)

In [2]:
classes = utils.get_imagenet_classes()
default_weights = utils.get_default_weights()
models_classif = utils.get_class_model_names()
models_ = utils.get_models_ensamble(models)
labels, index = get_coarse_arrays()

In [3]:
def generate_list(length, specified_positions, weights):
    weights = weights[0]
    output_list = [0] * length
    for i, pos in enumerate(specified_positions):
        output_list[pos] = weights[i]
    return output_list

def extract_and_normalize(original_probs, positions):
    # Extract values at specified positions
    extracted_values = original_probs[:, positions]
    # Sum of extracted values
    sum_values = torch.sum(extracted_values)
    # Normalize extracted values
    weights = extracted_values / sum_values
    return weights.tolist()

def extract_values(original_list, specified_indexes):
    # Use list comprehension to filter out values not corresponding to specified indexes
    extracted_values = [original_list[i] for i in range(len(original_list)) if i not in specified_indexes]
    return extracted_values

In [4]:
def randomize_phase_torch(image_tensor):
    # Convert torch tensor to NumPy array
    image_np = image_tensor.detach().numpy()

    # Separate color channels
    red_channel = image_np[:, 0, :, :]
    green_channel = image_np[:, 1, :, :]
    blue_channel = image_np[:, 2, :, :]

    # Compute FFT for each color channel
    red_fft = np.fft.fft2(red_channel, axes=(-2, -1))
    green_fft = np.fft.fft2(green_channel, axes=(-2, -1))
    blue_fft = np.fft.fft2(blue_channel, axes=(-2, -1))

    # Randomize phases for each color channel
    red_phases = np.angle(red_fft)
    green_phases = np.angle(green_fft)
    blue_phases = np.angle(blue_fft)

    randomized_g_phases = np.exp(1j * np.random.uniform(-np.pi, np.pi, red_phases.shape))
    randomized_green_phases = np.exp(1j * np.random.uniform(-np.pi, np.pi, green_phases.shape))
    randomized_blue_phases = np.exp(1j * np.random.uniform(-np.pi, np.pi, blue_phases.shape))

    # Combine magnitudes with randomized phases for each color channel
    randomized_red_fft = np.abs(red_fft) * randomized_g_phases
    randomized_green_fft = np.abs(green_fft) * randomized_g_phases
    randomized_blue_fft = np.abs(blue_fft) * randomized_g_phases

    # Compute the inverse FFT for each color channel
    randomized_red_channel = np.fft.ifft2(randomized_red_fft, axes=(-2, -1))
    randomized_green_channel = np.fft.ifft2(randomized_green_fft, axes=(-2, -1))
    randomized_blue_channel = np.fft.ifft2(randomized_blue_fft, axes=(-2, -1))

    # Take the abs to get the image for each color channel
    randomized_red_channel = np.abs(randomized_red_channel)
    randomized_green_channel = np.abs(randomized_green_channel)
    randomized_blue_channel = np.abs(randomized_blue_channel)

    # Combine color channels into RGB tensor
    randomized_image = np.stack((randomized_red_channel, randomized_green_channel, randomized_blue_channel), axis=1)
    randomized_image_plot = torch.from_numpy(randomized_image)

    print(f'Norma: {torch.norm(image_tensor)}, \nNorma: {torch.norm(randomized_image_plot)}')

    return randomized_image_plot

In [5]:
def ifgsm_attack(input, epsilon, data_grad, control = True, mode ='flip'):
    iter = int(min([epsilon + 4, epsilon * 1.25]))  # Number of iterations

    alpha = 1
    pert_out = input.clone().detach()

    for i in range(iter):
        pert_out = pert_out + (alpha / 255) * data_grad.sign()

        if torch.norm((pert_out - input), p=float('inf')) > epsilon / 255:
            break

    adv_pert = (pert_out - input)
    adv_pert_max = torch.max(adv_pert)
    adv_pert_min = torch.min(adv_pert)
    adv_pert_plot = (adv_pert - adv_pert_min)/(adv_pert_max - adv_pert_min)
    normalization = torch.mean(pert_out - input)

    pert_out_plot = pert_out

    if control == True:
        if mode == 'flip':
                       
                control_pert_lr = torch.flip(adv_pert_plot, [3]) #left - right
                control_pert_tb = torch.flip(adv_pert_plot, [2]) #top - bottom
                transposed_image = adv_pert_plot.transpose(2, 3)
                control_pert_diag = torch.flip(transposed_image, [2, 3]) #diagonal

                mse_right_left = F.mse_loss(input, control_pert_lr).item()
                mse_top_bottom = F.mse_loss(input, control_pert_tb).item()
                mse_diagonal = F.mse_loss(input, control_pert_diag).item()

                mse_values = {"Top-bottom": mse_top_bottom, "Diagonal": mse_diagonal, "Right-left": mse_right_left}
                max_mse_flipped_type = max(mse_values, key=mse_values.get)
                print(mse_values)
                                
                if max_mse_flipped_type == "Top-bottom":
                    control_pert_plot = control_pert_tb
                    control_pert = torch.flip(adv_pert, [3])
                    print('Control: Top Bottom')
                    control_image = control_pert + input

                elif max_mse_flipped_type == "Diagonal":
                    control_pert_plot = control_pert_diag
                                        
                    transposed_image = adv_pert.transpose(2, 3)
                    control_pert = torch.flip(transposed_image, [2, 3])
                    print('Control: Diagonal')
                    control_image = control_pert + input

                else:
                    control_pert_plot = control_pert_lr
                    control_pert = torch.flip(adv_pert, [2])
                    print('Control: Left Right')
                    control_image = control_pert + input


        elif mode == 'rndm':
                control_pert_plot =  randomize_phase_torch(adv_pert_plot)
                control_image = control_pert_plot*epsilon/255 + input
                control_image = control_image.float()

                print(adv_pert)
                print(control_pert_plot)

        return pert_out_plot, adv_pert_plot, control_image, control_pert_plot 
    else:
        return pert_out_plot, adv_pert_plot

In [14]:
def ifgsm_attack_alternative(input, epsilon, data_grad, control = True, mode ='flip'):
    iter = 10  # Number of iterations

    alpha = epsilon/iter
    pert_out = input.clone().detach()

    for i in range(iter):
        pert_out = pert_out + (alpha / 255) * data_grad.sign()
        pert_out = torch.clamp(pert_out, 0, 1)
        eta = torch.clamp(pert_out - input, -epsilon/255, epsilon/255)
        pert_out = torch.clamp(input + eta, 0, 1)

    adv_pert = (pert_out - input)
    adv_pert_max = torch.max(adv_pert)
    adv_pert_min = torch.min(adv_pert)
    adv_pert_plot = (adv_pert - adv_pert_min)/(adv_pert_max - adv_pert_min)
    normalization = torch.mean(pert_out - input)

    pert_out_plot = pert_out

    if control == True:
        if mode == 'flip':
                       
                control_pert_lr = torch.flip(adv_pert_plot, [3]) #left - right
                control_pert_tb = torch.flip(adv_pert_plot, [2]) #top - bottom
                transposed_image = adv_pert_plot.transpose(2, 3)
                control_pert_diag = torch.flip(transposed_image, [2, 3]) #diagonal

                mse_right_left = F.mse_loss(input, control_pert_lr).item()
                mse_top_bottom = F.mse_loss(input, control_pert_tb).item()
                mse_diagonal = F.mse_loss(input, control_pert_diag).item()

                mse_values = {"Top-bottom": mse_top_bottom, "Diagonal": mse_diagonal, "Right-left": mse_right_left}
                max_mse_flipped_type = max(mse_values, key=mse_values.get)
                print(mse_values)
                                
                if max_mse_flipped_type == "Top-bottom":
                    control_pert_plot = control_pert_tb
                    control_pert = torch.flip(adv_pert, [3])
                    print('Control: Top Bottom')
                    control_image = control_pert + input

                elif max_mse_flipped_type == "Diagonal":
                    control_pert_plot = control_pert_diag
                                        
                    transposed_image = adv_pert.transpose(2, 3)
                    control_pert = torch.flip(transposed_image, [2, 3])
                    print('Control: Diagonal')
                    control_image = control_pert + input

                else:
                    control_pert_plot = control_pert_lr
                    control_pert = torch.flip(adv_pert, [2])
                    print('Control: Left Right')
                    control_image = control_pert + input


        elif mode == 'rndm':
                control_pert_plot =  randomize_phase_torch(adv_pert_plot)
                control_image = control_pert_plot*epsilon/255 + input
                control_image = control_image.float()

                print(adv_pert)
                print(control_pert_plot)
                
        return pert_out_plot, adv_pert_plot, control_image, control_pert_plot 
    else:
        return pert_out_plot, adv_pert_plot

In [7]:
def make_up_case(img, epsilon, models, weights, original_output, c_index, c_classes, output_dir):

    print('Generating Attack to Make the original output higher')
    transformed_image = weights(img).unsqueeze(0)
    original_image = transformed_image.clone()
    loss = nn.CrossEntropyLoss()
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    # Calculate ensemble predictions
    ensemble_outputs = []
    transformed_image.requires_grad = True
    for model in models:
        model.eval()
    output = model(normalize(transformed_image))
    ensemble_outputs.append(output)

    ensemble_outputs = torch.stack(ensemble_outputs)
    ensemble_mean_output = torch.mean(ensemble_outputs, dim=0)

    # Get original predictions
    original_output = ensemble_mean_output.clone()

    for model in models:
        model.eval()
    
    target = original_output.max(1)[1]   #maximize original output
    cost = -loss(original_output, target)

    for model in models:
        model.zero_grad()

    #loss.backward()
    grad = torch.autograd.grad(cost, transformed_image)[0]

    # Generate adversarial image
    perturbed_image, adv_pert = ifgsm_attack(transformed_image, epsilon, grad, control = False)

    perturbed_output = torch.mean(torch.stack([model(normalize(perturbed_image)) for model in models]), dim=0)
    perturbed_probs = torch.softmax(perturbed_output, dim=1)

    # Compute coarse category scores
    scores_perturbed = torch.zeros(size= (len(c_index),))

    for i in np.arange(len(c_index)): #Iterate in the coarse classes

        c = c_index[i] #coarse class indexs

        c_values_p = perturbed_probs[:, c]
        sum_p = torch.sum(c_values_p)
        scores_perturbed[i] = sum_p


    #Eliminate values of probs (perturbed and original)
    coarse_idx = np.hstack(c_index) #index to eliminate

    pert_coarse_scores = eliminate_elements_torch(perturbed_probs[0], coarse_idx)

    new_classes = [string for idx, string in enumerate(classes) if idx not in coarse_idx]

    #Concat new scores and new list of classes
    pert_coarse_scores = torch.cat((pert_coarse_scores,  scores_perturbed), dim = 0)

    new_classes = new_classes + c_classes

    sorted_probs_p, sorted_indices_p = torch.sort(pert_coarse_scores, descending=True)
    sorted_classes_p = [new_classes[i] for i in sorted_indices_p]

            
        # Sort predictions
    perturbed_top_classes = perturbed_probs[0].topk(1000)
    perturbed_dict = {classes[idx.item()]: prob.item() for idx, prob in zip(perturbed_top_classes.indices, perturbed_top_classes.values)}


    #sorting values 
    sorted_probs_p, sorted_indices_p = torch.sort(pert_coarse_scores, descending=True)
    sorted_classes_p = [new_classes[i] for i in sorted_indices_p]
        
    #define dicts with probs

    probs_p_scores = {}
    for key, value in zip(sorted_classes_p, sorted_probs_p):
        probs_p_scores[key] = value.item() 


    results = []
    results.append({'perturbed': perturbed_dict, 'pert_coarse_scores': probs_p_scores})

    counter = 1
    perturbed_image = perturbed_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
    perturbed_image = np.clip(perturbed_image, 0,1)
    perturbed_image = (perturbed_image * 255).astype('uint8')

    adv_pert = adv_pert.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
    adv_pert =  np.clip(adv_pert, 0, 1)
    adv_pert = (adv_pert * 255).astype('uint8')

    while True:
            # Generate folder name
        folder_name = f"images_folder_{counter}_epsilon_{epsilon}"
        folder_path = os.path.join(output_dir, folder_name)

        # Check if folder already exists
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            break
        else:
            counter += 1
        

    atk_path = os.path.join(folder_path, 'atk.png')
    Image.fromarray(perturbed_image).resize((720,720)).save(atk_path)
    pert_path = os.path.join(folder_path, 'pert.png')
    Image.fromarray(adv_pert).resize((720,720)).save(pert_path)

    with gzip.open(os.path.join(folder_path, 'results.json.gz'), 'wt') as f:
                json.dump(results, f)
       

In [31]:
def attack(image_folder, models, weights, epsilon, classes, targeted = False, t = '0', coarse_class = 'nada', num_classes=1000, graph=False, folder=False, sorted = True,  attack = 'iFGSM',save_path = 0, make_upper = True, output_dir_up = '', control = True):
    """Test function to generate adversarial images and obtain predictions using an ensemble of models."""

    c_classes, c_index = get_coarse_arrays() 


    for model in models:
        model.eval()

    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

    # Prepare output directory
    if folder:
        if save_path == 0:
            if targeted == False:
                output_dir = f"ensamble-attack_ensamble-c_epsilon-{epsilon}-untargeted"
            else:
                output_dir = f"ensamble-attack_ensamble-c_epsilon-{epsilon}-targeted"
            
            os.makedirs(output_dir, exist_ok=True)
        else:
            output_dir = save_path

    results = []
    control = []

    print('Iterating over images in folder\n\n')
    
    for i, image_name in enumerate(os.listdir(image_folder)):
        image_path = os.path.join(image_folder, image_name)
        print(f'\n\nImage {i + 1}/{len(os.listdir(image_folder))}--------------------------------------------------------\nImage name:{image_name}')

        # Load and transform image
        img = Image.open(image_path)
        transformed_image = weights(img).unsqueeze(0)
        original_image = transformed_image.clone()

        # Calculate ensemble predictions
        ensemble_outputs = []
        transformed_image.requires_grad = True
        for model in models:
            model.eval()
            try:
                output = model(normalize(transformed_image))
                ensemble_outputs.append(output)

            except RuntimeError as e:
                if "output with shape [1, 1, 256, 256] doesn't match the broadcast shape [1, 3, 256, 256]" in str(e):
                    print("Encountered specific RuntimeError. Moving to next element.")
                    continue

        ensemble_outputs = torch.stack(ensemble_outputs)
        ensemble_mean_output = torch.mean(ensemble_outputs, dim=0)

        # Get original predictions
        original_output = ensemble_mean_output.clone()
        original_probs = torch.softmax(original_output, dim=1)

        # Calculate loss
        loss = nn.CrossEntropyLoss()

        if targeted == True:
            if type(t) == str:
                target = original_output.max(1)[1]   #maximize original output
                cost = -loss(original_output, target)
            elif type(t) == int:
                target = torch.tensor([t])           #maximize target t
                cost = -loss(original_output, target)

            elif type(t) == np.ndarray or type(t) == list:
                w = extract_and_normalize(original_probs, t)
                probs_t = generate_list(1000, t, w)
                target = torch.tensor([probs_t])           
                cost = -loss(original_output, target)
        else:
            if type(t) != str:
                non_t = extract_values(np.arange(1000), t)
                w = extract_and_normalize(original_probs, non_t)
                probs_t = generate_list(1000, non_t, w)
                target = torch.tensor([probs_t])           
                cost = -loss(original_output, target)
            else:
                target = original_output.max(1)[1]
                cost = loss(original_output, target)  #minimize original output

        #Calculate grad
        for model in models:
            model.zero_grad()

        #loss.backward()
        grad = torch.autograd.grad(cost, transformed_image)[0]

        if control == True:
            perturbed_image, adv_pert, control_image, control_pert = ifgsm_attack_alternative(transformed_image, epsilon, grad, control = True)
        else:
            perturbed_image, adv_pert = ifgsm_attack_alternative(transformed_image, epsilon, grad, control = False)
        # Get perturbed predictions

        perturbed_output = torch.mean(torch.stack([model(normalize(perturbed_image)) for model in models]), dim=0)
        perturbed_probs = torch.softmax(perturbed_output, dim=1)

        if control == True:
            control_output = torch.mean(torch.stack([model(normalize(control_image)) for model in models]), dim=0)
            control_probs = torch.softmax(control_output, dim=1)


        # Compute coarse category scores
        #print('Fine Class --> Coarse class')

        scores_original = torch.zeros(size= (len(c_index),))
        scores_perturbed = torch.zeros(size= (len(c_index),))
        if control == True:
            scores_control = torch.zeros(size= (len(c_index),))

        for i in np.arange(len(c_index)): #Iterate in the coarse classes

            c = c_index[i] #coarse class indexs

            c_values = original_probs[:, c]
            sum = torch.sum(c_values)
            scores_original[i] = sum

            c_values_p = perturbed_probs[:, c]
            sum_p = torch.sum(c_values_p)
            scores_perturbed[i] = sum_p

            if control == True:
                c_values_c = control_probs[:, c]
                sum_c = torch.sum(c_values_c)
                scores_control[i] = sum_c

        #Eliminate values of probs (perturbed and original)
        coarse_idx = np.hstack(c_index) #index to eliminate
        coarse_scores = eliminate_elements_torch(original_probs[0], coarse_idx)
        pert_coarse_scores = eliminate_elements_torch(perturbed_probs[0], coarse_idx)
        if control == True:
            ctrl_coarse_scores = eliminate_elements_torch(control_probs[0], coarse_idx)

        new_classes = [string for idx, string in enumerate(classes) if idx not in coarse_idx]

        #Concat new scores and new list of classes
        coarse_scores = torch.cat((coarse_scores, scores_original), dim = 0)
        pert_coarse_scores = torch.cat((pert_coarse_scores,  scores_perturbed), dim = 0)
        if control == True:
            ctrl_coarse_scores = torch.cat((ctrl_coarse_scores,  scores_control), dim = 0)

        new_classes = new_classes + c_classes

        sorted_probs_p, sorted_indices_p = torch.sort(pert_coarse_scores, descending=True)
        sorted_classes_p = [new_classes[i] for i in sorted_indices_p]

        if folder:
            if type(t) != str:
                if len(t) > 1:
                    a = targeted == True
                    b = coarse_class != sorted_classes_p[0]
                    c = targeted == False
                    d = coarse_class == sorted_classes_p[0]
                    if  (a and b) or (c and d):
                        continue

            else:
                a = targeted == True
                b = t != sorted_classes_p[0]
                c = targeted == False
                d = t == sorted_classes_p[0]
                print(t, '----', sorted_classes_p[0])
                if  (a and b) or (c and d):
                    continue


        if make_upper == True:
            make_up_case(img, epsilon, models, weights, original_output, c_index, c_classes, output_dir_up)

        # Sort predictions
        original_top_classes = original_probs[0].topk(num_classes)
        perturbed_top_classes = perturbed_probs[0].topk(num_classes)
        if control == True:
            control_top_classes = control_probs[0].topk(num_classes)

        original_dict = {classes[idx.item()]: prob.item() for idx, prob in zip(original_top_classes.indices, original_top_classes.values)}
        perturbed_dict = {classes[idx.item()]: prob.item() for idx, prob in zip(perturbed_top_classes.indices, perturbed_top_classes.values)}
        if control == True:
            control_dict = {classes[idx.item()]: prob.item() for idx, prob in zip(control_top_classes.indices, control_top_classes.values)}

        #sorting values 
        sorted_probs, sorted_indices = torch.sort(coarse_scores, descending=True)
        sorted_classes = [new_classes[i] for i in sorted_indices]

        sorted_probs_p, sorted_indices_p = torch.sort(pert_coarse_scores, descending=True)
        sorted_classes_p = [new_classes[i] for i in sorted_indices_p]
        
        if control == True:
            sorted_probs_c, sorted_indices_c = torch.sort(ctrl_coarse_scores, descending=True)
            sorted_classes_c = [new_classes[i] for i in sorted_indices_c]

        #define dicts with probs
        probs_scores = {}
        for key, value in zip(sorted_classes, sorted_probs):
            probs_scores[key] = value.item() 

        probs_p_scores = {}
        for key, value in zip(sorted_classes_p, sorted_probs_p):
            probs_p_scores[key] = value.item() 

        if control == True:
            probs_c_scores = {}
            for key, value in zip(sorted_classes_c, sorted_probs_c):
                probs_c_scores[key] = value.item() 

        results.append({'original': original_dict, 'perturbed': perturbed_dict, 'coarse_scores': probs_scores, 'pert_coarse_scores': probs_p_scores})
        if control == True:
            control.append({'ctrl': control_dict, 'ctrl_scores': probs_c_scores})

        #print(probs_c_scores)

        # Save adversarial image
        if folder:
            counter = 1

            perturbed_image = perturbed_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
            perturbed_image = np.clip(perturbed_image, 0,1)
            perturbed_image = (perturbed_image * 255).astype('uint8')

            adv_pert = adv_pert.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
            adv_pert =  np.clip(adv_pert, 0, 1)
            adv_pert = (adv_pert * 255).astype('uint8')

            if control == True:
                control_image = control_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
                control_image = np.clip(control_image, 0, 1)
                control_image = (control_image * 255).astype('uint8')

                control_pert = control_pert.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
                control_pert = np.clip(control_pert, 0,1 )
                control_pert = (control_pert * 255).astype('uint8')

            while True:
                # Generate folder name
                folder_name = f"images_folder_{counter}_epsilon_{epsilon}"
                folder_path = os.path.join(output_dir, folder_name)

                # Check if folder already exists
                if not os.path.exists(folder_path):
                    os.makedirs(folder_path)
                    break
                else:
                    counter += 1
                    if counter > 165:
                        print(f'There are {165} images in the folder: MAX NUMBER REACHED.')
                        return

                

            atk_path = os.path.join(folder_path, 'atk.png')
            Image.fromarray(perturbed_image).resize((720,720)).save(atk_path)
            pert_path = os.path.join(folder_path, 'pert.png')
            Image.fromarray(adv_pert).resize((720,720)).save(pert_path)

            if control == True:
                control_pert_path = os.path.join(folder_path, 'control_pert.png')
                Image.fromarray(control_pert).resize((720,720)).save(control_pert_path, optimize=True)
                control_path = os.path.join(folder_path, 'control.png')
                Image.fromarray(control_image).resize((720,720)).save(control_path, optimize=True)

            with gzip.open(os.path.join(folder_path, 'results.json.gz'), 'wt') as f:
                json.dump(results, f)
                
            if control == True:
                with gzip.open(os.path.join(folder_path, 'control.json.gz'), 'wt') as f:
                    json.dump(control, f)

            if (a & b):
                print('SAVED IMAGE. Equal Target as Top Perturbed')
            elif (c & d):
                print('SAVED IMAGE. Different Top Perturbed from Original')


        print(f'Epsilon: {epsilon}')
        if targeted == True:
            if type(t) == str:
                print(f'Targeted Attack. Maximize Original Output\nOriginal prob: {probs_scores[sorted_classes[0]]},\nPerturbed prob: {probs_p_scores[sorted_classes[0]]}\nControl prob: {probs_c_scores[sorted_classes[0]]}')
            elif len(t) == 1:
                print(f'Targeted Attack = {classes[t[0]]}\nOriginal prob target: {probs_scores[classes[t[0]]]},\nPerturbed prob target: {probs_p_scores[classes[t[0]]]}\nControl prob target: {probs_c_scores[classes[t[0]]]}')
            elif len(t) > 1:
                print(f'Targeted Attack = {coarse_class}\nOriginal prob target: {probs_scores[coarse_class]},\nPerturbed prob target: {probs_p_scores[coarse_class]},\nPerturbed Top class & Prob: {sorted_classes_p[0]} & {sorted_probs_p[0]}')
                if control == True:
                    print(f'\nControl prob target: {probs_c_scores[coarse_class]}')
        else:
            print(f'Untargeted Attack. Minimize Original Output\nOriginal prob target: {probs_scores[sorted_classes[0]]},\nPerturbed prob target: {probs_p_scores[sorted_classes[0]]},\nPerturbed Top class & Prob: {sorted_classes_p[0]} & {sorted_probs_p[0]}')

        if graph:
            if folder != True:
                perturbed_image = perturbed_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
                adv_pert_plot = adv_pert.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
                if control == True:
                    control_image = control_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0)
                control_pert = control_pert.squeeze().detach().cpu().numpy().transpose(1, 2, 0)

            f, axs = plt.subplots(1, 3, figsize=(17, 5))
            axs[0].imshow(original_image.squeeze().detach().cpu().numpy().transpose(1, 2, 0))
            axs[0].set_title(f"Original: {max(original_dict, key=original_dict.get)} - {np.round(max(original_dict.values()), 3)}" + "\n" + f"Original: {max(probs_scores, key=probs_scores.get)} - {np.round(max(probs_scores.values()), 3)}", fontsize=7)
            axs[0].set_axis_off()

            axs[1].imshow(np.clip(perturbed_image, 0, 1))
            axs[1].set_title(f"Perturbed: {max(perturbed_dict, key=perturbed_dict.get)} - {np.round(max(perturbed_dict.values()), 3)}" + "\n" + f"Perturbed: {max(probs_p_scores, key=probs_p_scores.get)} - {np.round(max(probs_p_scores.values()), 3)}", fontsize=7)
            axs[1].set_axis_off()

            axs[2].imshow(np.clip(control_image, 0, 1))
            axs[2].set_title(f"Control Image: {max(control_dict, key=control_dict.get)} - {np.round(max(control_dict.values()), 3)}" + "\n" + f"Control: {max(probs_c_scores, key=probs_c_scores.get)} - {np.round(max(probs_c_scores.values()), 3)}", fontsize=7)
            axs[2].set_axis_off() 

            plt.subplots_adjust(wspace=0.5)  # Add space between the plots
            plt.show()

            plt.subplot(1, 2, 1)
            plt.imshow(adv_pert_plot)
            plt.title(f"Perturbation", fontsize=7)
            plt.axis('off') 

            plt.subplot(1, 2, 2)
            plt.imshow(np.clip(control_pert, 0, 1))
            plt.title(f"Perturbation - Control", fontsize=7)
            plt.axis('off') 

            plt.subplots_adjust(wspace=0.5)  # Add space between the plots
            plt.show()

    return results

In [11]:
pizza_f = r'C:\Users\Usuario\Documents\Trini\Facultad\Tesis\Código Personal\image_net\class_images_image_net\00963'

In [12]:
classes[963]

'pizza, pizza pie'

In [32]:
for eps in [16]:
    pizza_up = rf'C:\Users\Usuario\Documents\Trini\Facultad\Tesis\Código Personal\Results\Exp 2\Pizza\T up\Epsilon {eps}'
    pizza_down = rf'C:\Users\Usuario\Documents\Trini\Facultad\Tesis\Código Personal\Results\Exp 2\Pizza\T down\Epsilon {eps}'
    attack(pizza_f, models_, default_weights, eps, classes, targeted=False, t = classes[963], graph= False, folder = True, save_path=pizza_down, output_dir_up=pizza_up)

Iterating over images in folder




Image 1/659--------------------------------------------------------
Image name:000855800129055.jpg
pizza, pizza pie ---- tray
Generating Attack to Make the original output higher
Epsilon: 2
Untargeted Attack. Minimize Original Output
Original prob target: 0.5771474838256836,
Perturbed prob target: 0.0038948131259530783,
Perturbed Top class & Prob: tray & 0.279943585395813


Image 2/659--------------------------------------------------------
Image name:004181525192045.jpg
pizza, pizza pie ---- pizza, pizza pie


Image 3/659--------------------------------------------------------
Image name:004282416650661.jpg
pizza, pizza pie ---- pizza, pizza pie


Image 4/659--------------------------------------------------------
Image name:005860669879614.jpg
pizza, pizza pie ---- pizza, pizza pie


Image 5/659--------------------------------------------------------
Image name:007756893710414.jpg
pizza, pizza pie ---- pizza, pizza pie


Image 6/659----------------

KeyboardInterrupt: 